In [ ]:
import aiohttp
import asyncio
import nest_asyncio
import pandas as pd

from mainnet_launch.constants import *
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
import numpy as np

# allow nested loops if you ever run this inside Jupyter/debugger

nest_asyncio.apply()

_rate_limit = asyncio.Semaphore(10)
_session: aiohttp.ClientSession | None = None


async def get_session() -> aiohttp.ClientSession:
    global _session
    if _session is None or _session.closed:
        _session = aiohttp.ClientSession()
    return _session


async def fetch_swap_quote(
    chain_id: int,
    sell_token: str,
    buy_token: str,
    sell_amount: int,
    system_name: str = "gen3",
    slippage_bps: int = 50,
    include_sources: str = "",
    exclude_sources: str = "Bebop",
    sell_all: bool = True,
    timeout_ms: int = None,
    transfer_to_caller: bool = True,
) -> dict:
    async with _rate_limit:
        url = "https://swaps-pricing.tokemaklabs.com/swap-quote-v2"
        payload = {
            "chainId": chain_id,
            "systemName": system_name,
            "slippageBps": slippage_bps,
            "taker": DEAD_ADDRESS,
            "sellToken": sell_token,
            "buyToken": buy_token,
            "sellAmount": sell_amount,
            "includeSources": include_sources,
            "excludeSources": exclude_sources,
            "sellAll": sell_all,
            "timeoutMS": str(timeout_ms) if timeout_ms is not None else "",
            "transferToCaller": str(transfer_to_caller),
        }

        session = await get_session()
        async with session.post(url, json=payload) as resp:
            try:
                resp.raise_for_status()
                data = await resp.json()
                data.update(payload)
            except Exception as e:
                data = {"error": str(e), **payload}
        return data


async def fetch_quote_size_df(
    token_orms: list[Tokens], sizes: list[float], autopool: AutopoolConstants
) -> pd.DataFrame:

    tasks = []

    for size in sizes:
        for t in token_orms:

            tasks.append(
                fetch_swap_quote(
                    chain_id=t.chain_id,
                    sell_token=t.token_address,
                    buy_token=autopool.base_asset,
                    sell_amount=int(size * 10**t.decimals),
                )
            )
    quotes = await asyncio.gather(*tasks)
    quote_df = pd.DataFrame.from_records(quotes)
    return quote_df


async def fetch_quote_df(
    token_orms: list[Tokens], base_sell_amount: float, autopool: AutopoolConstants
) -> pd.DataFrame:
    # launch all requests in parallel
    quotes = await asyncio.gather(
        *(
            fetch_swap_quote(
                chain_id=t.chain_id,
                sell_token=t.token_address,
                buy_token=autopool.base_asset,
                sell_amount=int(base_sell_amount * 10**t.decimals),
            )
            for t in token_orms
        )
    )
    return pd.DataFrame.from_records(quotes)


async def quote_at_scale():
    autopool = SONIC_USD
    token_orms = get_full_table_as_orm(Tokens, where_clause=Tokens.chain_id == autopool.chain.chain_id)
    tokens_df = pd.DataFrame.from_records([t.to_record() for t in token_orms])

    # first pass to filter out errors
    initial_df = await fetch_quote_df(token_orms, 100, autopool)
    merged = pd.merge(initial_df, tokens_df, how="left", left_on="sellToken", right_on="token_address")
    tokens_we_can_get_quotes_for = merged[merged["error"].isna()]["token_address"].unique().tolist()
    quote_able_tokens_orm = [t for t in token_orms if t.token_address in tokens_we_can_get_quotes_for]

if __name__ == "__main__":
    asyncio.run(quote_at_scale())


In [11]:
import quotes_at_scale
import ast

import numpy as np
import plotly.express as px
import pandas as pd
import plotly.io as pio

pio.templates.default = None

df = pd.read_csv(
    "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/mainnet_launch/data_fetching/quotes/sonicUSD_scale_quotes.csv"
)

df["min_buy_scaled"] = df["minBuyAmount"].apply(lambda x: int(x) / 1e6 if x > 0 else np.nan)
df["buy_scaled"] = df["buyAmount"].apply(lambda x: int(x) / 1e6 if x > 0 else np.nan)

# scale sellAmount by its own per-row decimals
df["sell_amount_scaled"] = df.apply(lambda row: int(row["sellAmount"]) / (10 ** row["decimals"]), axis=1)

df["buy_ratio"] = df["buy_scaled"] / df["sell_amount_scaled"]
df["min_buy_ratio"] = df["min_buy_scaled"] / df["sell_amount_scaled"]

price_df = df.pivot(columns="symbol", index="sell_amount_scaled", values="buy_scaled")
ratio_df = df.pivot(columns="symbol", index="sell_amount_scaled", values="buy_ratio")
ratio_df

symbol,USDT,eUSDC.e-2,eUSDC.e-3,escUSD-1,escUSD-2,frxUSD,scUSD,wstkscUSD
sell_amount_scaled,,,,,,,,
100.0,1.000803,1.006791,1.015986,1.009898,1.011188,0.999768,0.999143,1.005940
1000.0,1.000799,1.006791,1.015986,1.009889,1.011179,NaN,0.999134,1.005793
10000.0,1.000758,1.006791,1.015986,1.009855,1.011146,NaN,0.999101,1.005535
50000.0,1.000662,1.006791,1.015986,1.009831,1.011122,NaN,0.999078,1.005377
100000.0,1.000593,1.006791,1.015986,1.009808,1.011099,NaN,0.999055,1.005273
200000.0,1.000493,1.006791,1.015986,NaN,NaN,NaN,NaN,1.005048
300000.0,NaN,1.006791,1.015986,1.009717,NaN,NaN,NaN,1.004750
400000.0,1.000368,1.006791,1.015986,NaN,NaN,NaN,NaN,NaN
500000.0,1.000316,1.006791,1.015986,NaN,NaN,NaN,NaN,0.031336


In [ ]:
# not symetrical down,

# all the near the same time?

# not really certian there

In [12]:
px.scatter(ratio_df, title="sonic_USD_ratio", log_x=True)

In [ ]:
df[df["symbol"] == "escUSD1"]

In [ ]:
price_df

In [ ]:
# quotes fail, for > 100 frxUSD

In [ ]:
df.columns

In [2]:
df = pd.read_csv("quotes_records2.csv")
df["fullQuoteDetails_parsed"] = df["fullQuoteDetails"].apply(ast.literal_eval)


def extract_block(row):

    if "data" in row["fullQuoteDetails_parsed"]:
        return int(row["fullQuoteDetails_parsed"]["data"]["blockNumber"])
    elif "quote" in row["fullQuoteDetails_parsed"]:
        return int(row["fullQuoteDetails_parsed"]["quote"]["blockNumber"])


df["block"] = df.apply(extract_block, axis=1)
df

,Unnamed: 0,aggregatorName,internal,asyncSwapper,minBuyAmount,buyAmount,expiration,tx,fullQuoteDetails,chainId,...,sellAll,timeoutMS,transferToCaller,token_address,chain_id,symbol,name,decimals,fullQuoteDetails_parsed,block
0,0,OpenOcean,False,0x98b55E07156895814d2503036dC635316F9C9502,19890258868,20010320792,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'code': 200, 'data': {'inToken': {'address': ...",146,...,True,NaN,True,0x6047828dc181963ba44974801FF68e538dA5eaF9,146,USDT,Tether USD,6,"{'code': 200, 'data': {'inToken': {'address': ...",35728501.0
1,1,4626,True,0xAb9F53c36B9c82AbBe397742d4A0aD75Df33241c,20035040051,20135718644,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': False}",146,...,True,NaN,True,0x3D9e5462A940684073EED7e4a13d19AE0Dcd13bc,146,eUSDC.e-2,EVK Vault eUSDC.e-2,6,"{'redeem': True, 'swap': False}",NaN
2,2,OpenOcean,False,0x98b55E07156895814d2503036dC635316F9C9502,19859484604,19979360768,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'code': 200, 'data': {'inToken': {'address': ...",146,...,True,NaN,True,0xd3DCe716f3eF535C5Ff8d041c1A41C3bd89b97aE,146,scUSD,Sonic USD,6,"{'code': 200, 'data': {'inToken': {'address': ...",35728501.0
3,3,4626,True,0xAb9F53c36B9c82AbBe397742d4A0aD75Df33241c,20071817167,20192975018,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': True, 'underlyingQuot...",146,...,True,NaN,True,0xeEb1DC1Ca7ffC5b54aD1cc4c1088Db4E5657Cb6c,146,escUSD-1,EVK Vault escUSD-1,6,"{'redeem': True, 'swap': True, 'underlyingQuot...",NaN
4,4,4626,True,0xAb9F53c36B9c82AbBe397742d4A0aD75Df33241c,20098725574,20220045849,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': True, 'underlyingQuot...",146,...,True,NaN,True,0xB38D431e932fEa77d1dF0AE0dFE4400c97e597B8,146,escUSD-2,EVK Vault escUSD-2,6,"{'redeem': True, 'swap': True, 'underlyingQuot...",NaN
5,5,4626,True,0xAb9F53c36B9c82AbBe397742d4A0aD75Df33241c,20217972860,20319570713,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': False}",146,...,True,NaN,True,0x196F3C7443E940911EE2Bb88e019Fd71400349D9,146,eUSDC.e-3,EVK Vault eUSDC.e-3,6,"{'redeem': True, 'swap': False}",NaN
6,6,OpenOcean,False,0x98b55E07156895814d2503036dC635316F9C9502,19993652889,20114338922,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'code': 200, 'data': {'inToken': {'address': ...",146,...,True,NaN,True,0x9fb76f7ce5FCeAA2C42887ff441D46095E494206,146,wstkscUSD,Wrapped stkscUSD,6,"{'code': 200, 'data': {'inToken': {'address': ...",35728501.0
7,7,OpenOcean,False,0x98b55E07156895814d2503036dC635316F9C9502,19868900217,19988833217,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'code': 200, 'data': {'inToken': {'address': ...",146,...,True,NaN,True,0x80Eede496655FB9047dd39d9f418d5483ED600df,146,frxUSD,Frax USD,18,"{'code': 200, 'data': {'inToken': {'address': ...",35728505.0
8,0,OpenOcean,False,0x98b55E07156895814d2503036dC635316F9C9502,19890258868,20010320792,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'code': 200, 'data': {'inToken': {'address': ...",146,...,True,NaN,True,0x6047828dc181963ba44974801FF68e538dA5eaF9,146,USDT,Tether USD,6,"{'code': 200, 'data': {'inToken': {'address': ...",35728721.0
9,1,4626,True,0xAb9F53c36B9c82AbBe397742d4A0aD75Df33241c,20035042264,20135720868,9999999999,{'from': '0x0000000000000000000000000000000000...,"{'redeem': True, 'swap': False}",146,...,True,NaN,True,0x3D9e5462A940684073EED7e4a13d19AE0Dcd13bc,146,eUSDC.e-2,EVK Vault eUSDC.e-2,6,"{'redeem': True, 'swap': False}",NaN


In [3]:
df["symbol"].value_counts()

symbol
USDT         3
eUSDC.e-2    3
scUSD        3
escUSD-1     3
escUSD-2     3
eUSDC.e-3    3
wstkscUSD    3
frxUSD       3
Name: count, dtype: int64

In [4]:
# did something change this minute with the sell able toekns?

In [6]:
df["min_buy_scaled"] = df["minBuyAmount"].apply(lambda x: int(x) / 1e6)
df["buy_scaled"] = df["buyAmount"].apply(lambda x: int(x) / 1e6)
df["sell_amount_scaled"] = df.apply(lambda row: int(row["sellAmount"]) / (10 ** row["decimals"]), axis=1)

In [7]:
df[["symbol", "min_buy_scaled", "buy_scaled", "sell_amount_scaled"]]

,symbol,min_buy_scaled,buy_scaled,sell_amount_scaled
0,USDT,19890.258868,20010.320792,20000.0
1,eUSDC.e-2,20035.040051,20135.718644,20000.0
2,scUSD,19859.484604,19979.360768,20000.0
3,escUSD-1,20071.817167,20192.975018,20000.0
4,escUSD-2,20098.725574,20220.045849,20000.0
5,eUSDC.e-3,20217.972860,20319.570713,20000.0
6,wstkscUSD,19993.652889,20114.338922,20000.0
7,frxUSD,19868.900217,19988.833217,20000.0
8,USDT,19890.258868,20010.320792,20000.0
9,eUSDC.e-2,20035.042264,20135.720868,20000.0


In [8]:
import plotly.express as px

px.scatter(df, x=df["block"], y="min_buy_scaled", color="symbol")

In [ ]:
px.scatter(df, x=df.index, y="buy_scaled", color="symbol")